<a href="https://colab.research.google.com/github/shreyas-sreedhar/gen-ai-csye7380/blob/main/Assignment-04/Assignment04_Shreyas_S_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir -p '/content/drive/MyDrive/GenAI_2/images'

In [4]:
from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Conv2D, Conv2DTranspose, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# We'll start by rewriting the architecture for both the generator and discriminator networks
# using convolutional layers and adjusting the code to work with the CIFAR-10 dataset.



class GAN:
    def __init__(self):
        # Input shape for CIFAR-10 images (32x32x3)
        self.img_shape = (32, 32, 3)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates images
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model, we only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # Combined model (stacked generator and discriminator)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(256 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 256)))
        model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2DTranspose(3, kernel_size=4, strides=1, padding='same', activation='tanh'))

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=200):
        # Load CIFAR-10 data and use one class (e.g., 'deer', class 4)
        (X_train, y_train), (_, _) = cifar10.load_data()
        X_train = X_train[y_train.flatten() == 4]

        # Rescale images from [0,255] to [-1,1]
        X_train = X_train / 127.5 - 1.0

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            #  Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            #  Train Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)

            # Print the progress
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")

            # Save generated image samples at save_interval
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig(f"/content/drive/MyDrive/GenAI_2/images/cifar10_{epoch}.png")
        plt.close()


In [7]:

if __name__ == '__main__':
    gan = GAN()

    gan.train(epochs=100, batch_size=32, save_interval=20)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
0 [D loss: 0.679500937461853, acc.: 68.75%] [G loss: [array(0.68397176, dtype=float32), array(0.68397176, dtype=float32), array(0.65625, dtype=float32)]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1 [D loss: 0.6897320747375488, acc.: 52.34375%] [G loss: [array(0.69110703, dtype=float32), array(0.69110703, dtype=float32), array(0.453125, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
2 [D loss: 0.6920084953308105, acc.: 44.68750059604645%] [G loss: [array(0.6935706, dtype=float32), array(0.6935706, dtype=float32), array(0.40625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
3 [D loss: 0.6934541463851929, acc.: 41.43415093421936%] [G loss: [array(0.69519395, dtype=float32), array(0.69519395, dtype=float32), array(0.38671875, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
4 [D loss: 0.6956453323364258, acc.: 39.91319537162781%] [G loss: [array(0.69778425, dtype=float32), array(0.69778425, dtype=float32), array(0.378125, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
5 [D loss: 0.6986672878265381, acc.: 38.11553120613098%] [G loss: [array(0.70104617, dtype=float32), arr

In [3]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, LeakyReLU, Conv2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist


In [4]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.n_classes = 10

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim + self.n_classes,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        noise_shape = (self.latent_dim + self.n_classes,)

        model = Sequential()
        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        img_shape = self.img_shape

        model = Sequential()
        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):
        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # One-hot encode class labels
        y_train = to_categorical(y_train, num_classes=self.n_classes)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
            labels = y_train[idx]

            noise = np.random.normal(0, 1, (half_batch, self.latent_dim))
            noise_with_labels = np.concatenate((noise, labels), axis=1)

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise_with_labels)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

            # Calculate average loss and accuracy
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            d_loss_value = d_loss[0]
            d_acc = d_loss[1]

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            labels = to_categorical(np.random.randint(0, self.n_classes, batch_size), num_classes=self.n_classes)
            noise_with_labels = np.concatenate((noise, labels), axis=1)

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.ones((batch_size, 1))

            # Train the generator
            g_loss = self.combined.train_on_batch(noise_with_labels, valid_y)

            # Plot the progress
            print(f"{epoch} [D loss: {d_loss_value}, acc.: {d_acc*100}%] [G loss: {g_loss}]")

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

        # Save weights after training
        self.save_weights("final")

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        labels = to_categorical(np.random.randint(0, self.n_classes, r * c), num_classes=self.n_classes)
        noise_with_labels = np.concatenate((noise, labels), axis=1)

        gen_imgs = self.generator.predict(noise_with_labels)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        plt.savefig(f"/content/drive/MyDrive/GenAI_1/images/mnist_{epoch}.png")
        plt.close()

    # def save_weights(self, suffix):
    #     """Save weights for the generator and discriminator."""
    #     self.generator.save_weights(f"/content/drive/MyDrive/GenAI_1/images/generator_weights_{suffix}.h5")
    #     self.discriminator.save_weights(f"/content/drive/MyDrive/GenAI_1/images/discriminator_weights_{suffix}.h5")

    # def load_weights(self, suffix):
    #     """Load weights for the generator and discriminator."""
    #     self.generator.load_weights(f"/content/drive/MyDrive/GenAI_1/images/generator_weights_{suffix}.h5")
    #     self.discriminator.load_weights(f"/content/drive/MyDrive/GenAI_1/images/discriminator_weights_{suffix}.h5")

if __name__ == '__main__':
    gan = GAN()
    # suffix = "final"  # Use a suffix to load weights
    # try:
    #     gan.load_weights(suffix)  # Load weights if they exist
    #     print(f"Loaded weights for suffix '{suffix}'.")
    # except Exception as e:
    #     print(f"Could not load weights for suffix '{suffix}': {e}")
    #     print("Starting training from scratch.")

    gan.train(epochs=1001, batch_size=32, save_interval=200)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

...

Could not load weights for suffix 'final': [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/GenAI_1/images/generator_weights_final.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Starting training from scratch.
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.6683915257453918, acc.: 45.3125%] [G loss: [array(0.6827743, dtype=float32), array(0.6827743, dtype=float32), array(0.6827743, dtype=float32), array(0.40625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


1 [D loss: 0.6499322652816772, acc.: 52.60416269302368%] [G loss: [array(0.6538147, dtype=float32), array(0.6538147, dtype=float32), array(0.6538147, dtype=float32), array(0.53125, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2 [D loss: 0.6385149359703064, acc.: 58.22916626930237%] [G loss: [array(0.64649343, dtype=float32), array(0.64649343, dtype=float32), array(0.64649343, dtype=float32), array(0.5520833, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
3 [D loss: 0.6386643648147583, acc.: 58.03571343421936%] [G loss: [array(0.64588994, dtype=float32), array(0.64588994, dtype=float32), array(0.64588994, dtype=float32), array(0.5625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4 [D loss: 0.6566299200057983, acc.: 56.35416507720947%] [G loss: [array(0.6634685, dtype=float32), array(0.6634685, dtype=float32), array(0.6634685, dtype=float32), array(0.54375, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
5 [D loss: 0.6618335247039795, acc.: 55.516099929

ValueError: The filename must end in `.weights.h5`. Received: filepath=/content/drive/MyDrive/GenAI_1/images/generator_weights_final.h5